In [1]:
from openfast_io.FAST_output_reader import FASTOutputFile
from scipy import fft
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
# Extracting the data from the OpenFAST output file
data = FASTOutputFile('/Users/mchetan/Desktop/nrel/projects/3-STABLE/tools/echo/tests/simplifiedTurbine/fa-nacelle/simpleTurbine.out')

dataDF = data.toDataFrame()

In [3]:

# Get time column and determine the 101s mark index
time_column = dataDF['Time_[s]']
trim_index = np.argmin(np.abs(time_column - 101.0))

# Trim the data
trimmed_dataDF = dataDF.iloc[trim_index:].copy()


In [13]:
# Perform FFT on the trimmed data with Plotly visualizations

# Create subplot with 2 rows and 1 column
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=['Magnitude Spectrum (Frequencies < 10Hz Only) - Log Scale', 
                                   'Phase Spectrum (Frequencies < 10Hz Only)'],
                    vertical_spacing=0.15,
                    shared_xaxes=True)

# Define colors for different channels (accessibility-friendly palette)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', 
          '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aec7e8', '#ffbb78',
          '#98df8a', '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#c7c7c7']

# Track the channels for legend
channels_list = [
    'B1N005_TVxr_[m/s]', 
    'B2N005_TVxr_[m/s]', 
    'B3N005_TVxr_[m/s]',
    'B1N005_TVyr_[m/s]', 
    'B2N005_TVyr_[m/s]', 
    'B3N005_TVyr_[m/s]',
    ]

for idx, channels in enumerate(channels_list):
    # Perform FFT
    x = fft.fft(np.array(trimmed_dataDF[channels]))

    magnitude = np.abs(x)  # Magnitude of each frequency component
    phase = np.angle(x)  # Phase of each frequency component

    fs = 1000.0  # Sampling frequency (using the existing variable)

    # Get the frequency axis
    freq = fft.fftfreq(x.shape[-1], d=1/fs)

    # Get the positive frequency components (first half of the FFT output)
    n = len(x)
    pos_freq = freq[:n//2]  # Only positive frequencies
    pos_magnitude = magnitude[:n//2]  # Corresponding magnitudes
    pos_phase = phase[:n//2]  # Corresponding phases

    # Filter for frequencies < 5Hz
    below_10hz_mask = pos_freq < 5
    freq_below_10hz = pos_freq[below_10hz_mask]
    magnitude_below_10hz = pos_magnitude[below_10hz_mask]
    phase_below_10hz = pos_phase[below_10hz_mask]

    # Add magnitude trace (with log scale)
    fig.add_trace(
        go.Scatter(
            x=freq_below_10hz,
            y=magnitude_below_10hz,
            mode='lines',
            name=channels,
            line=dict(color=colors[idx]),
            legendgroup=channels,  # Group by channel for legend
            hovertemplate='Frequency: %{x:.3f} Hz<br>Magnitude: %{y:.6f}<extra></extra>',
        ),
        row=1, col=1
    )

    # Add phase trace
    fig.add_trace(
        go.Scatter(
            x=freq_below_10hz,
            y=np.unwrap(phase_below_10hz),
            mode='lines',
            name=channels,
            line=dict(color=colors[idx]),
            legendgroup=channels,  # Group by channel for legend
            showlegend=False,  # Don't duplicate legend entries
            hovertemplate='Frequency: %{x:.3f} Hz<br>Phase: %{y:.6f} rad<extra></extra>',
        ),
        row=2, col=1
    )

# Update layout for log scale in magnitude plot
fig.update_yaxes(type='log', row=1, col=1)

# Update overall layout
fig.update_layout(
    height=800, 
    width=1000,
    title_text='FFT Analysis of Blade Velocity Components',
    xaxis_title='Frequency (Hz)',
    xaxis2_title='Frequency (Hz)',
    yaxis_title='Magnitude (log scale)',
    yaxis2_title='Phase (radians)',
    legend_title='Channel',
    hoverlabel=dict(
        bgcolor='white',
        font_size=16,
        font_family='Rockwell'
    ),
    # Add hover mode for vertical line on hover
    hovermode='x unified'
)

# Add grid lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# Show the figure
fig.show()

In [ ]:
# Create a more advanced Plotly FFT analysis with additional features
# This example adds a slider to control the frequency range

def plot_fft_with_slider(dataDF, channels_list, max_freq=50):
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=['Magnitude Spectrum - Log Scale', 
                                       'Phase Spectrum'],
                        vertical_spacing=0.15)

    colors = ['blue', 'red', 'green', 'purple', 'orange']
    
    for idx, channel in enumerate(channels_list):
        # Perform FFT
        x = fft.fft(np.array(dataDF[channel]))
        magnitude = np.abs(x)
        phase = np.angle(x)
        
        fs = 1 / 0.001  # Sampling frequency
        freq = fft.fftfreq(x.shape[-1], d=1/fs)
        
        # Get positive frequencies
        n = len(x)
        pos_freq = freq[:n//2]
        pos_magnitude = magnitude[:n//2]
        pos_phase = phase[:n//2]
        
        # Filter by max frequency
        freq_mask = pos_freq < max_freq
        filtered_freq = pos_freq[freq_mask]
        filtered_magnitude = pos_magnitude[freq_mask]
        filtered_phase = pos_phase[freq_mask]
        
        # Add traces
        fig.add_trace(
            go.Scatter(
                x=filtered_freq,
                y=filtered_magnitude,
                mode='lines',
                name=channel,
                line=dict(color=colors[idx % len(colors)])
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=filtered_freq,
                y=np.unwrap(filtered_phase),
                mode='lines',
                name=channel,
                line=dict(color=colors[idx % len(colors)]),
                showlegend=False
            ),
            row=2, col=1
        )
    
    # Update layout
    fig.update_yaxes(type='log', row=1, col=1)
    
    fig.update_layout(
        height=800, 
        width=1000,
        title_text='FFT Analysis with Frequency Range Control',
        xaxis_title='Frequency (Hz)',
        xaxis2_title='Frequency (Hz)',
        yaxis_title='Magnitude (log scale)',
        yaxis2_title='Phase (radians)',
        legend_title='Channel',
        hoverlabel=dict(
            bgcolor='white',
            font_size=14,
        ),
        updatemenus=[
            dict(
                buttons=[
                    dict(label='Linear Scale',
                         method='update',
                         args=[{}, {'yaxis': {'type': 'linear'}}]),
                    dict(label='Log Scale',
                         method='update',
                         args=[{}, {'yaxis': {'type': 'log'}}]),
                ],
                direction='down',
                pad={'r': 10, 't': 10},
                showactive=True,
                x=0.1,
                xanchor='left',
                y=1.15,
                yanchor='top'
            )
        ],
        sliders=[dict(
            active=10,
            currentvalue={"prefix": "Max Frequency: ", "suffix": " Hz"},
            pad={"t": 50},
            steps=[
                dict(method='update',
                     args=[{'visible': [True if i < (max_freq*j/50) else False for i in range(len(channels_list)*2)]}],
                     label=str(j)) for j in range(1, 51, 5)
            ])
        )]
    )
    
    # Add grid lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    return fig

# Example usage:
channels_to_analyze = ['B1N005_TVxr_[m/s]', 'B2N005_TVxr_[m/s]', 'B3N005_TVxr_[m/s]']
fft_fig = plot_fft_with_slider(trimmed_dataDF, channels_to_analyze, max_freq=10)
fft_fig.show()